In [2]:
import numpy as np
import matplotlib.pyplot as plt
# If you need to import from a local utils.py, uncomment and adjust the following lines:
import sys
import os
import jax.numpy as jnp
# Construct the full path to the folder
folder_path = r'C:\Users\Petrb\Desktop\DTU\3rdSemester\02477_BAYESIAN_MACHINE_LEARNING'

# Add the folder to the Python path
sys.path.append(folder_path)

# Now you can import the utils module
from utils import *

# Utility Function and Calculation 

The core calculation is done with the line: `expected_util = phat @ U`.

For a single prediction point (one row of phat), the expected utility of predicting class j is the sum of the utilities for predicting class j across all possible true classes, weighted by the probabilities of those true classes.
Mathematically, for each prediction point i and class j:

`expected_util[i,j] = sum(phat[i,k] * U[k,j] for k in range(K))`

When you perform the matrix multiplication phat @ U, you're calculating this sum for every prediction point and every possible class prediction simultaneously.

#### A Simple Example
Imagine we have a binary classification problem (K=2) with:

**Utility matrix U** = `[[1, -100], [-10, 1]]`

`U[0,0] = 1:`    Utility of correctly predicting class 0 \
`U[0,1] = -100:` Utility of incorrectly predicting class 1 when true class is 0 \
`U[1,0] = -10:`  Utility of incorrectly predicting class 0 when true class is 1 \
`U[1,1] = 1:`    Utility of correctly predicting class 1 



For a single prediction point with probabilities phat = [0.7, 0.3]

70% chance of being class 0
30% chance of being class 1



The expected utility calculation would be:

For predicting class 0: 0.7 × 1 + 0.3 × (-10) = 0.7 - 3 = -2.3 \
For predicting class 1: 0.7 × (-100) + 0.3 × 1 = -70 + 0.3 = -69.7

Since -2.3 > -69.7, we would choose to predict class 0 to maximize expected utility.

What the Function Returns \
The function returns expected_util, a matrix of shape [P × K], where each element [i,j] represents the expected utility of predicting class j for the ith prediction point.

In [ ]:
def compute_expected_utility(U, phat):
    """
    Computes the expected utility for a multi-class classification problem with K classes,
    given a utility matrix U and posterior predictive probabilities phat.

    # Mathematical Equation:
    For each prediction point i and each possible action/class j:
        expected_util[i, j] = sum_k phat[i, k] * U[k, j]
    where:
        - phat[i, k] is the predicted probability of class k for point i
        - U[k, j] is the utility of predicting class j when the true class is k

    # Arguments:
    U      : np.ndarray or jnp.ndarray, shape [K, K]
             Utility matrix, where U[k, j] is the utility of predicting class j when the true class is k.
             A utility matrix of shape [K × K], where K is the number of classes. Each element U[i,j] 
             represents the utility (or reward) of predicting class j when the true class is i.
             
            
    phat   : np.ndarray or jnp.ndarray, shape [P, K]
             Posterior predictive probabilities for P prediction points and K classes.
             osterior predictive probabilities of shape [P × K], where P is the number 
             of prediction points. Each row of phat contains the probabilities that a 
             particular instance belongs to each of the K classes.

    # Returns:
    expected_util : np.ndarray or jnp.ndarray, shape [P, K]
                    Expected utility for each class (action) for each prediction point.

    # Example:
    >>> U = np.array([[1, -1], [0, 2]])  # shape [2, 2]
    >>> phat = np.array([[0.7, 0.3], [0.4, 0.6]])  # shape [2, 2]
    >>> compute_expected_utility(U, phat)
    array([[0.7*1 + 0.3*0, 0.7*-1 + 0.3*2],
           [0.4*1 + 0.6*0, 0.4*-1 + 0.6*2]])
    array([[0.7, 0.7*(-1) + 0.3*2],
           [0.4, 0.4*(-1) + 0.6*2]])

    # Input shapes:
    U:     [K, K]
    phat:  [P, K]

    # Output shape:
    expected_util: [P, K]
    """

    # Print shapes for debugging
    print(f"U.shape: {U.shape}")      # Should be [K, K]
    print(f"phat.shape: {phat.shape}")  # Should be [P, K]

    # Compute expected utility using matrix multiplication
    # phat: [P, K], U: [K, K] -> result: [P, K]
    expected_util = phat @ U

    # Check dimensions and return
    assert expected_util.shape == phat.shape, (
        f'The variable expected_util was expected to have shape {phat.shape}, '
        f'but the actual shape was {expected_util.shape}. Please check your code.'
    )
    return expected_util

## Exampe 1 

In [19]:
# Define the utility matrix from your cancer example
# U[true class, predicted class]
# Format: [
#   [U₀₀, U₀₁],
#   [U₁₀, U₁₁]
# ]
U = np.array([
    [1, -100],    # Utilities when true class is 0 (not cancer)
    [-10, 1]      # Utilities when true class is 1 (cancer)
])

###########################################################################
# Define some example posterior probabilities for 3 different patients
# Each row represents P(y=0|x) and P(y=1|x) for a patient
# Format: [
#   [P(y=0|x_1), P(y=1|x_1)],
#   [P(y=0|x_2), P(y=1|x_2)],
#   [P(y=0|x_3), P(y=1|x_3)]
# ]
phat = np.array([
    [0.95, 0.05],  # Patient 1: 95% chance of not having cancer
    [0.85, 0.15],  # Patient 2: 85% chance of not having cancer
    [0.70, 0.30]   # Patient 3: 70% chance of not having cancer
])

###########################################################################

# Call the function
expected_util = compute_expected_utility(U, phat)

# The output from the debug print statements would be:
# (2, 2)      # Shape of U
# (3, 2)      # Shape of phat

print("Expected utilities:")
print(expected_util)

# Find the optimal prediction for each patient by selecting the class
# with the maximum expected utility
optimal_predictions = np.argmax(expected_util, axis=1)

# Print the results in a more readable format
for i, (probs, utils, pred) in enumerate(zip(phat, expected_util, optimal_predictions)):
    print(f"\nPatient {i+1}:")
    print(f"  Probability of no cancer: {probs[0]:.2f}, Probability of cancer: {probs[1]:.2f}")
    print(f"  Expected utility if we predict 'no cancer': {utils[0]:.2f}")
    print(f"  Expected utility if we predict 'cancer': {utils[1]:.2f}")
    print(f"  Optimal prediction: {'No cancer' if pred == 0 else 'Cancer'}")


U.shape: (2, 2)
phat.shape: (3, 2)
Expected utilities:
[[  0.45 -94.95]
 [ -0.65 -84.85]
 [ -2.3  -69.7 ]]

Patient 1:
  Probability of no cancer: 0.95, Probability of cancer: 0.05
  Expected utility if we predict 'no cancer': 0.45
  Expected utility if we predict 'cancer': -94.95
  Optimal prediction: No cancer

Patient 2:
  Probability of no cancer: 0.85, Probability of cancer: 0.15
  Expected utility if we predict 'no cancer': -0.65
  Expected utility if we predict 'cancer': -84.85
  Optimal prediction: No cancer

Patient 3:
  Probability of no cancer: 0.70, Probability of cancer: 0.30
  Expected utility if we predict 'no cancer': -2.30
  Expected utility if we predict 'cancer': -69.70
  Optimal prediction: No cancer


# Example 2 

In [22]:
U = np.array([
    [2, 1],    # Utilities when true class is 0 (not cancer)
    [1, 2]      # Utilities when true class is 1 (cancer)
])

y_hat_1 = 0.129
y_hat_0 = 1 - y_hat_1


phat = np.array([
    [y_hat_0, y_hat_1]  # Patient 1: 95% chance of not having cancer
])

# Call the function
expected_util = compute_expected_utility(U, phat)

# The output from the debug print statements would be:
# (2, 2)      # Shape of U
# (3, 2)      # Shape of phat

print("Expected utilities:")
print(expected_util)

# Find the optimal prediction for each patient by selecting the class
# with the maximum expected utility
optimal_predictions = np.argmax(expected_util, axis=1)

# Print the results in a more readable format
for i, (probs, utils, pred) in enumerate(zip(phat, expected_util, optimal_predictions)):
    print(f"\nPatient {i+1}:")
    print(f"  Probability of no cancer: {probs[0]:.2f}, Probability of cancer: {probs[1]:.2f}")
    print(f"  Expected utility if we predict 'no cancer': {utils[0]:.2f}")
    print(f"  Expected utility if we predict 'cancer': {utils[1]:.2f}")
    print(f"  Optimal prediction: {'No cancer' if pred == 0 else 'Cancer'}")

U.shape: (2, 2)
phat.shape: (1, 2)
Expected utilities:
[[1.871 1.129]]

Patient 1:
  Probability of no cancer: 0.87, Probability of cancer: 0.13
  Expected utility if we predict 'no cancer': 1.87
  Expected utility if we predict 'cancer': 1.13
  Optimal prediction: No cancer
